In [ ]:
import pypsa
import yaml
import cartopy
import sys
import re
import os

import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from itertools import product
from matplotlib.lines import Line2D
from matplotlib.patches import FancyArrowPatch
from vresutils.costdata import annuity

PATH = "../../../playgrounds/pr/pypsa-eur-sec/"

sys.path.append(os.path.join(PATH, "scripts/"))
from plot_summary import rename_techs

plt.style.use(["bmh", "matplotlibrc"])
xr.set_options(display_style="html")

%matplotlib inline

In [ ]:
CLUSTERS = 181
LV_OPTS = "Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10"
OUTPUT = "../results/graphics/"

H2_SCENARIOS = PATH + "results/20211218-181-h2"
DEC_SCENARIOS = PATH + "results/20211218-181-decentral"
LV_SCENARIOS = PATH + "results/20211218-181-lv"
ONW_SCENARIOS = PATH + "results/20211218-181-onw"
GAS_SCENARIOS = PATH + "results/20211218-181-gas"

In [ ]:
with open(PATH + "config.yaml") as file:
    config = yaml.safe_load(file)

In [ ]:
def rename_techs_tyndp(tech):
    tech = rename_techs(tech)
    if "heat pump" in tech or "resistive heater" in tech:
        return "power-to-heat"
    elif tech in ["H2 Electrolysis", "methanation", "helmeth", "H2 liquefaction"]:
        return "power-to-gas"
    elif "H2 pipeline" in tech:
        return "H2 pipeline"
    elif tech == "H2":
        return "H2 storage"
    elif tech in ["OCGT", "CHP", "gas boiler", "H2 Fuel Cell"]:
        return "gas-to-power/heat"
    # elif "solar" in tech:
    #    return "solar"
    elif tech == "Fischer-Tropsch":
        return "power-to-liquid"
    elif "offshore wind" in tech:
        return "offshore wind"
    elif "CC" in tech or "sequestration" in tech:
        return "CCS"
    elif tech == "oil" or tech == "gas":
        return "fossil oil and gas"
    else:
        return tech

In [ ]:
preferred_order = pd.Index(
    [
        "transmission lines",
        "electricity distribution grid",
        "fossil oil and gas",
        "hydroelectricity",
        "hydro reservoir",
        "run of river",
        "pumped hydro storage",
        "solid biomass",
        "biogas",
        "onshore wind",
        "offshore wind",
        "offshore wind (AC)",
        "offshore wind (DC)",
        "solar PV",
        "solar thermal",
        "solar rooftop",
        "solar",
        "building retrofitting" "ground heat pump",
        "air heat pump",
        "heat pump",
        "resistive heater",
        "power-to-heat",
        "gas-to-power/heat",
        "CHP",
        "OCGT",
        "gas boiler",
        "gas",
        "natural gas",
        "helmeth",
        "methanation",
        "power-to-gas",
        "power-to-H2",
        "H2 pipeline",
        "H2 liquefaction",
        "H2 storage",
        "hydrogen storage",
        "power-to-liquid",
        "battery storage",
        "hot water storage",
        "CO2 sequestration",
        "CCS",
    ]
)

In [ ]:
def load_decentral():
    costs = pd.read_csv(
        DEC_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
    )

    costs = costs.xs(
        (str(CLUSTERS), "2030"), level=["cluster", "planning_horizon"], axis=1
    )

    def parse_index(c):
        lv = c[0]

        match = re.search(r"onwind\+p([0-9.]*)", c[1])
        onw = 100.0 if match is None else 100 * float(match.groups()[0])

        h2 = "no H2 grid" if "noH2network" in c[1] else "H2 grid"

        return (lv, onw, h2)

    costs.columns = pd.MultiIndex.from_tuples(
        [parse_index(c) for c in costs.columns], names=["lv", "onw", "h2"]
    )

    df = costs.groupby(level=2).sum().div(1e9)

    df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

    df = df.xs(100, level="onw", axis=1, drop_level=False)

    df.sum() / df.sum().min()

    to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
    print(to_drop)
    df.drop(to_drop, inplace=True)

    order = preferred_order.intersection(df.index).append(
        df.index.difference(preferred_order)
    )
    df = df.loc[order]

    tech_colors = config["plotting"]["tech_colors"]
    colors = [tech_colors[i] for i in df.index]

    df.columns = df.columns.get_level_values(2)
    df.columns.name = ""

    return df

In [ ]:
def load_h2(rename=True):
    costs = pd.read_csv(
        H2_SCENARIOS + f"/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
    )

    costs = costs.xs(
        (str(CLUSTERS), "2030"), level=["cluster", "planning_horizon"], axis=1
    )

    costs.columns = pd.MultiIndex.from_tuples(
        [parse_index(c) for c in costs.columns], names=["lv", "onw", "h2"]
    )

    df = costs.groupby(level=2).sum().div(1e9)

    if rename:
        df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

    to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
    print(to_drop)
    df.drop(to_drop, inplace=True)

    order = preferred_order.intersection(df.index).append(
        df.index.difference(preferred_order)
    )
    df = df.loc[order]

    return df

In [ ]:
def parse_index(c):
    lv = c[0]

    match = re.search(r"onwind\+p([0-9.]*)", c[1])
    onw = 100.0 if match is None else 100 * float(match.groups()[0])

    h2 = "no H2 grid" if "noH2network" in c[1] else "H2 grid"

    return (lv, onw, h2)

In [ ]:
def load_h2_capacities():
    df = pd.read_csv(
        H2_SCENARIOS + f"/csvs/capacities.csv", header=[0, 1, 2, 3], index_col=[0, 1]
    )

    df = df.xs((str(CLUSTERS), "2030"), level=["cluster", "planning_horizon"], axis=1)

    df.columns = pd.MultiIndex.from_tuples(
        [parse_index(c) for c in df.columns], names=["lv", "onw", "h2"]
    )

    df = df.groupby(level=1).sum()

    return df

## Line Volume Sensitivity

In [ ]:
costs = pd.read_csv(
    LV_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
)

In [ ]:
costs = costs[str(CLUSTERS)].rename(lambda x: float(x), axis=1, level=0)

In [ ]:
df = (
    costs.xs(LV_OPTS, level="opt", axis=1)
    .xs("2030", level="planning_horizon", axis=1)
    .groupby(level=2)
    .sum()
    .div(1e9)
)

In [ ]:
df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

In [ ]:
df.sum()

In [ ]:
df.sum() - df.sum().max()

In [ ]:
(1 - df.sum() / df.sum().max()) * 100

In [ ]:
to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
to_drop = to_drop.difference(
    ["DAC", "hot water storage"]
)  # exclude dropping to align with onwind sensitivity...
df.drop(to_drop, inplace=True)

In [ ]:
order = preferred_order.intersection(df.index).append(
    df.index.difference(preferred_order)
)
df = df.loc[order]

In [ ]:
df.columns *= 100

In [ ]:
dec = load_decentral()["H2 grid"]
dec.name = "\nno power\ntransmission"
dec = pd.DataFrame(dec).T

In [ ]:
dec.sum().sum()

In [ ]:
fig, (ax0, ax1) = plt.subplots(
    1,
    2,
    figsize=(5, 3),
    sharey=True,
    gridspec_kw={"width_ratios": [1, 6], "wspace": 0.05},
)

to_plot = df.T.sort_index()

tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in df.index]

to_plot.plot.area(ax=ax1, stacked=True, linewidth=0, color=colors)

dec.plot.bar(ax=ax0, stacked=True, linewidth=0, legend=False, color=tech_colors)

handles, labels = ax1.get_legend_handles_labels()

handles.reverse()
labels.reverse()

ax1.set_xlim(100, 200)
ax1.set_xlabel("Power Grid Reinforcement Restriction\n[% relative to today's volume]")

ax0.set_ylim([0, 1000])
ax0.set_ylabel("System Cost\n[EUR billion per year]")
ax0.grid(axis="y")
ax1.grid(axis="y")

ax1.axvline(125, color="k", linestyle="--", linewidth=1)

ax1.text(127.5, 810, "TYNDP equivalent", size=11, color="k")

# legend on side
ax1.legend(handles, labels, ncol=2, frameon=False, bbox_to_anchor=(1.02, 1.03))

for i in ["top", "right", "left", "bottom"]:
    ax1.spines[i].set_visible(False)
    ax0.spines[i].set_visible(False)

# legend inside
# ax1.legend(handles, labels, ncol=3, frameon=False, bbox_to_anchor=(1.1, 1.48))

ax1.set_axisbelow(False)

ax0.tick_params(rotation=0, labelsize=10)
ax1.tick_params(rotation=0, labelsize=10)
ax1.set_xticks([100, 125, 150, 175, 200])

fig.savefig(OUTPUT + "lv-sensitivity.pdf", bbox_inches="tight")

## Onshore Wind Sensitivity

In [ ]:
costs = pd.read_csv(
    ONW_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
)

In [ ]:
costs = costs.xs(
    (str(CLUSTERS), "1.25", "2030"), level=["cluster", "lv", "planning_horizon"], axis=1
)

In [ ]:
costs.columns = [
    100 * float(re.search(r"onwind\+p([0-9.]*)", c).groups()[0]) for c in costs.columns
]

In [ ]:
df = costs.groupby(level=2).sum().div(1e9)

In [ ]:
df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

In [ ]:
df.sum()

In [ ]:
df.sum() - df.sum().max()

In [ ]:
(1 - df.sum() / df.sum().min()) * 100

In [ ]:
to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
print(to_drop)
df.drop(to_drop, inplace=True)

In [ ]:
order = preferred_order.intersection(df.index).append(
    df.index.difference(preferred_order)
)
df = df.loc[order]

In [ ]:
spec = load_h2().T

spec = spec.xs(("1.0", 0.0, "H2 grid"), level=["lv", "onw", "h2"])

spec.index = ["no grid\nexpansion\nno onshore wind"]

In [ ]:
(spec.sum().sum() - 886.040764) / 886.040764

In [ ]:
fig, (ax0, ax1) = plt.subplots(
    1,
    2,
    figsize=(5, 3),
    sharey=True,
    gridspec_kw={"width_ratios": [1, 6], "wspace": 0.05},
)

to_plot = df.T.sort_index()

tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in df.index]

to_plot.plot.area(ax=ax1, stacked=True, linewidth=0, color=colors)

spec.plot.bar(ax=ax0, stacked=True, linewidth=0, legend=False, color=tech_colors)

handles, labels = ax1.get_legend_handles_labels()

handles.reverse()
labels.reverse()

ax1.set_xlim(0, 100)
ax1.set_xlabel("Fraction of technical onshore\nwind potential available [%]")

ax0.set_ylim(0, 1000)
ax0.set_ylabel("System Cost\n[EUR billion per year]")
ax0.grid(axis="y")
ax1.grid(axis="y")

ax1.axvline(25, color="k", linestyle="--", linewidth=1)

ax1.text(27.5, 900, "compromise social potential", size=11, color="k")

# legend on side
ax1.legend(handles, labels, ncol=2, frameon=False, bbox_to_anchor=(1, 1.02))

for i in ["top", "right", "left", "bottom"]:
    ax1.spines[i].set_visible(False)
    ax0.spines[i].set_visible(False)

# legend inside
# ax.legend(handles, labels, ncol=3, frameon=False, bbox_to_anchor=(1.1, 1.48))

ax1.set_axisbelow(False)

ax0.tick_params(rotation=0, labelsize=10)
ax1.tick_params(rotation=0, labelsize=10)
ax1.set_xticks([0, 25, 50, 75, 100])

fig.savefig(OUTPUT + "onw-sensitivity.pdf", bbox_inches="tight")

## H2 Network Scenarios

In [ ]:
df = load_h2()

In [ ]:
df.sum().sort_values()

In [ ]:
dff = df.sum().unstack("h2")

In [ ]:
rel_advantage = (((dff["no H2 grid"] / dff["H2 grid"] - 1) * 100).round(1)).unstack().T
rel_advantage

In [ ]:
abs_advantage = ((dff["no H2 grid"] - dff["H2 grid"]).round().astype(int)).unstack().T
abs_advantage

In [ ]:
(df.sum() - df.sum().max()).sort_values().round()

In [ ]:
((1 - df.sum() / df.sum().max()) * 100).sort_values().round(1)

In [ ]:
x = df.sum().xs(100.0, level="onw")
xx = x / x.min()
xx.unstack().round(3)

In [ ]:
dff.xs(100.0, level="onw")

In [ ]:
df.filter(like="H2 pipeline", axis=0)

In [ ]:
dff.xs(0.0, level="onw")

In [ ]:
857.451 / 764.136

In [ ]:
936.4 / 898.3

In [ ]:
tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in df.index]

In [ ]:
colors = [
    "hydroelectricity",
    "onshore wind",
    "offshore wind",
    "solar PV",
    "solar thermal",
    "solar rooftop",
    "power-to-heat",
    "gas-to-power/heat",
    "power-to-gas",
    "power-to-liquid",
    "battery storage",
    "hot water storage",
    "CCS",
    "H2 storage",
    "SMR",
    "electricity distribution grid",
]

In [ ]:
color_map = {i: tech_colors[i] for i in colors}
color_map["HVDC transmission"] = "darkseagreen"
color_map["HVAC transmission"] = "rosybrown"
color_map["power-to-heat"] = "firebrick"

In [ ]:
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(10, 1.5))
handles = [mpatches.Patch(color=v, label=k) for k, v in color_map.items()]
ax.legend(handles=handles, ncol=6)
ax.axis("off")
plt.savefig(OUTPUT + "color_legend.pdf", bbox_inches="tight")

In [ ]:
xx = enumerate(df.columns.get_level_values("lv").unique()[::-1])
yy = enumerate(df.columns.get_level_values("onw").unique())

fig, axs = plt.subplots(2, 2, figsize=(5.5, 5.5), sharex=True, sharey=True)

kwargs = dict(stacked=True, color=colors, ylim=(0, 1100), legend=False)

for x, y in product(xx, yy):
    ax = axs[x[0], y[0]]

    toprow_kwargs = (
        dict(title=f"onshore wind\n{int(y[1])}% potential\n") if x[0] == 0 else {}
    )

    ylabel_value = "optimal" if x[1] == "opt" else f"{100 * float(x[1]) - 100:.{0}f}%"

    to_plot = df.xs((x[1], y[1]), axis=1, level=["lv", "onw"]).T.sort_index(
        ascending=True
    )
    to_plot.plot.bar(
        ax=ax,
        **kwargs,
        ylabel=f"line expansion\n{ylabel_value}\n\nbn€/a",
        **toprow_kwargs,
    )

    ax.set_xlabel("", rotation=0)

    ax.tick_params(labelrotation=0)

    ax.grid(axis="y")
    ax.title.set_size(11)

    # ax.patch.set_visible(False)

    for i in ["top", "right", "left", "bottom"]:
        ax.spines[i].set_visible(False)

    label = f"+ {rel_advantage.loc[y[1],x[1]]}%\n+ {abs_advantage.loc[y[1],x[1]]}"
    ax.text(0.31, 825, label, size=8.5, color="k")


handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
fig.legend(handles, labels, bbox_to_anchor=(1.4, 0.9))

plt.tight_layout()

plt.savefig(OUTPUT + "sensitivity-h2.pdf", bbox_inches="tight")

In [ ]:
df_new = df.xs(100, level="onw", axis=1)

xx = enumerate(df_new.columns.get_level_values("lv").unique()[::-1])
yy = enumerate(df_new.columns.get_level_values("h2").unique())

fig, axs = plt.subplots(2, 2, figsize=(4, 6), sharey=True)

plt.subplots_adjust(hspace=0.5, wspace=1)

kwargs = dict(stacked=True, color=colors, ylim=(0, 900), legend=False)

for x, y in product(xx, yy):
    ax = axs[x[0], y[0]]

    toprow_kwargs = (
        dict(
            title="with\nhydrogen grid\n"
            if y[1] == "H2 grid"
            else "without\nhydrogen grid\n"
        )
        if x[0] == 0
        else {}
    )

    ylabel = (
        "with power\ngrid expansion\n\nbn€/a"
        if x[1] == "opt"
        else "without power\ngrid expansion\n\nbn€/a"
    )

    to_plot = df_new.xs((x[1], y[1]), axis=1, level=["lv", "h2"]).T

    to_plot.plot.bar(
        ax=ax,
        ylabel=ylabel,
        **kwargs,
        **toprow_kwargs,
    )

    ax.set_xlabel("", rotation=0)
    ax.set_xticks([], [])

    ax.tick_params(labelrotation=0)

    ax.grid(axis="y")
    ax.title.set_size(11)

    ax.set_yticks(np.arange(0, 901, 100))

    # ax.patch.set_visible(False)

    for i in ["top", "right", "left", "bottom"]:
        ax.spines[i].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
fig.legend(handles, labels, bbox_to_anchor=(1.5, 0.9))

fig.text(0.45, 0.32, "+ 46\n+ 5.7%", fontsize=11.5)

fig.text(0.45, 0.68, "+ 31\n+ 4.1%", fontsize=11.5)

fig.text(0.28, 0.49, "+ 47\n+ 6.2%", fontsize=11.5)

fig.text(0.63, 0.49, "+ 62\n+ 7.8%", fontsize=11.5)

fig.text(0.45, 0.55, "+ 93\n+ 12.2%", fontsize=11.5, color="grey")


def add_arrow(ax0, ax1, pos0, pos1, **arrow_kwargs):
    ax0tr = ax0.transData  # Axis 0 -> Display
    ax1tr = ax1.transData  # Axis 1 -> Display
    figtr = fig.transFigure.inverted()  # Display -> Figure
    ptB = figtr.transform(ax0tr.transform(pos0))
    ptE = figtr.transform(ax1tr.transform(pos1))
    arrow = FancyArrowPatch(
        ptB,
        ptE,
        transform=fig.transFigure,  # Place arrow in figure coord system
        **arrow_kwargs,
    )
    fig.patches.append(arrow)


arrow_style = dict(arrowstyle="simple", mutation_scale=15, ec=None)

add_arrow(axs[0, 0], axs[0, 1], (0.5, 500), (-0.5, 500), fc="olivedrab", **arrow_style)

add_arrow(axs[1, 0], axs[1, 1], (0.5, 500), (-0.5, 500), fc="orange", **arrow_style)

add_arrow(axs[0, 0], axs[1, 0], (0, 0), (0, 900), fc="orange", **arrow_style)

add_arrow(axs[0, 1], axs[1, 1], (0, 0), (0, 900), fc="firebrick", **arrow_style)

add_arrow(axs[0, 0], axs[1, 1], (0.5, 0), (-0.5, 900), fc="lightgrey", **arrow_style)

# plt.tight_layout()

plt.savefig(OUTPUT + "sensitivity-h2-new.pdf", bbox_inches="tight")

### H2 vs Electricity Grid

In [ ]:
levels = ["lv", "onw", "h2"]

In [ ]:
scenA = df.xs(("1.0", 100.0, "H2 grid"), level=levels, axis=1)
scenB = df.xs(("opt", 100.0, "no H2 grid"), level=levels, axis=1)
scenC = df.xs(("1.0", 100.0, "no H2 grid"), level=levels, axis=1)
scenD = df.xs(("opt", 100.0, "H2 grid"), level=levels, axis=1)

In [ ]:
scenA.columns = ["grid expansion\nelectricity: no\nhydrogen: yes"]
scenB.columns = ["grid expansion\nelectricity: yes\nhydrogen: no"]
scenC.columns = ["grid expansion\nelectricity: no\nhydrogen: no"]
scenD.columns = ["grid expansion\nelectricity: yes\nhydrogen: yes"]

In [ ]:
to_plot = pd.concat([scenD, scenB, scenA, scenC], axis=1).T

In [ ]:
tsc = to_plot.sum(axis=1)

In [ ]:
base = tsc["grid expansion\nelectricity: yes\nhydrogen: no"]

In [ ]:
diff_rel = (100 * tsc / base - 100).round(1)[
    "grid expansion\nelectricity: no\nhydrogen: yes"
]

In [ ]:
diff_abs = (tsc - base).round(1)["grid expansion\nelectricity: no\nhydrogen: yes"]

In [ ]:
diff_abs

In [ ]:
tsc.values

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))

to_plot.plot.bar(
    ax=ax,
    stacked=True,
    color=colors,
    ylim=(0, 900),
    ylabel="total system cost [bn€/a]",
)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1, 1.02))

label = f"+ {diff_rel}%"
ax.text(1.28, 820, label, size=11, color="#444444")

label = f"+ {diff_abs} bn€/a"
ax.text(1.28, 860, label, size=11, color="k")

for i, v in enumerate(tsc.astype(int).values):
    ax.text(i - 0.07, v + 12, v, size=10, color="#444444")

ax.grid(axis="y")
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0, 901, 100), fontsize=11)

for i in ["top", "right", "left", "bottom"]:
    ax.spines[i].set_visible(False)

ax.axvline(0.5, color="k", linewidth=1.25, linestyle="--")
ax.axvline(2.5, color="k", linewidth=1.25, linestyle="--")

plt.tight_layout()

plt.savefig(OUTPUT + "h2-vs-elec-grid.pdf", bbox_inches="tight")

### Growing Exclusion

In [ ]:
kwargs = dict(level=["lv", "onw", "h2"], axis=1)

In [ ]:
costs = pd.read_csv(
    DEC_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
)

In [ ]:
costs = costs.xs((str(CLUSTERS), "2030"), level=["cluster", "planning_horizon"], axis=1)

In [ ]:
scenA = df.xs(("opt", 100.0, "H2 grid"), **kwargs)
scenB = df.xs(("1.0", 100.0, "H2 grid"), **kwargs)
scenC = df.xs(("1.0", 0.0, "H2 grid"), **kwargs)
scenD = df.xs(("1.0", 0.0, "no H2 grid"), **kwargs)

In [ ]:
scenA.columns = ["least-cost"]
scenB.columns = ["no grid expansion"]
scenC.columns = ["no grid expansion\nno onshore wind"]
scenD.columns = ["no grid expansion\nno onshore wind\nno hydrogen grid"]

In [ ]:
to_plot = pd.concat([scenA, scenB, scenC, scenD], axis=1).T

In [ ]:
to_plot.loc["today", "today"] = 700

order = to_plot.index[:-1].insert(0, to_plot.index[-1])
to_plot = to_plot.loc[order]

In [ ]:
to_plot.sum(axis=1) / to_plot.sum(axis=1)["least-cost"]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

to_plot.plot.bar(
    ax=ax,
    stacked=True,
    color=colors,
    ylim=(0, 1000),
    ylabel="total system cost [bn€/a]",
)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1, 1.05))

ax.grid(axis="y")
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0, 1001, 100), fontsize=11)

for i in ["top", "right", "left", "bottom"]:
    ax.spines[i].set_visible(False)

plt.tight_layout()

plt.savefig(OUTPUT + "growing-exclusion.pdf", bbox_inches="tight")

## Decentral Scenarios

In [ ]:
df = load_decentral()

In [ ]:
to_plot = df.T

In [ ]:
tsc = to_plot.sum(axis=1)

In [ ]:
diff_rel = (100 * tsc / tsc.min() - 100).round(1)[1]

In [ ]:
diff_abs = (tsc - tsc.min()).round(1)[1]

In [ ]:
to_plot

In [ ]:
tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in to_plot.columns]

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 5))

to_plot.plot.bar(
    ax=ax,
    stacked=True,
    color=colors,
    ylim=(0, 1000),
    ylabel="total system cost [bn€/a]",
)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1, 1.02))

label = f"+ {diff_rel}%"
ax.text(0.2, 920, label, size=11, color="#444444")

label = f"+ {diff_abs} bn€/a"
ax.text(0.2, 960, label, size=11, color="k")

ax.grid(axis="y")
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0, 1001, 100), fontsize=11)

for i in ["top", "right", "left", "bottom"]:
    ax.spines[i].set_visible(False)

plt.tight_layout()

plt.savefig(OUTPUT + "decentral.pdf", bbox_inches="tight")

## TWkm

In [ ]:
kind = "twkm"

twkm = pd.read_csv(f"../data/{kind}.csv", index_col=[0, 1])

twkm

data = pd.DataFrame()

data["Electricity network existing"] = twkm.filter(like="old", axis=1).sum(axis=1)
data["Electricity network new"] = twkm.filter(like="new", axis=1).sum(axis=1)
data["Hydrogen network retrofitted"] = twkm["H2 pipeline retrofitted"]
data["Hydrogen network new"] = twkm["H2 pipeline"]

In [ ]:
kind = "ewhkm"

twkm = pd.read_csv(f"../data/{kind}.csv", index_col=[0, 1])

twkm

data = pd.DataFrame()

data["Electricity network (HVAC)"] = twkm.filter(like="AC", axis=1).sum(axis=1)
data["Electricity network (HVDC)"] = twkm.filter(like="DC", axis=1).sum(axis=1)
data["Hydrogen network retrofitted"] = twkm["H2 pipeline retrofitted"]
data["Hydrogen network new"] = twkm["H2 pipeline"]

In [ ]:
nindex = ["cost-optimal\nexpansion", "no grid\nexpansion"]

In [ ]:
fig, ax = plt.subplots(
    1, 2, sharey=True, figsize=(4, 3.5), gridspec_kw={"width_ratios": [1.6, 1]}
)

kwargs = dict(stacked=True, edgecolor="k", width=0.3)

df = data.xs("yes", level="h2").filter(like="Electricity")
df.index = nindex
df.plot.bar(ax=ax[0], color=["#6c9459", "#a6be9b"], position=1, **kwargs)

dfh = data.xs("yes", level="h2").filter(like="Hydrogen")
dfh.index = nindex
dfh.plot.bar(ax=ax[0], color=["#904d84", "#f081dc"], position=0, **kwargs)

df2 = data.xs("no", level="h2")
df2.index = nindex[:2]
df2.plot.bar(
    ax=ax[1],
    color=["#6c9459", "#a6be9b"],
    legend=False,
    edgecolor="k",
    width=0.5,
    stacked=True,
)

if kind == "twkm":
    ax[0].set_ylabel("TWkm")
    ax[0].set_ylim([0, 800])

if kind == "ewhkm":
    ax[0].set_ylabel("EWhkm")
    ax[0].set_ylim([0, 4])

ax[0].set_xlim([-0.5, 1.5])
ax[0].set_xlabel("")
ax[0].set_title("H$_2$ network")
ax[1].set_title("no H$_2$ network")
ax[1].set_xlabel("")

ax[0].title.set_size(11)
ax[1].title.set_size(11)

ax[0].grid(axis="y")
ax[1].grid(axis="y")

for i in ["top", "right", "left", "bottom"]:
    ax[0].spines[i].set_visible(False)
    ax[1].spines[i].set_visible(False)

plt.tight_layout()

ax[0].legend(bbox_to_anchor=(1.45, 1.6), ncol=1)

plt.savefig(f"{OUTPUT}/{kind}.pdf", bbox_inches="tight")

In [ ]:
twkm.sum(axis=1)

In [ ]:
1 - 379 / 485

In [ ]:
twkm.filter(regex="(AC|DC)", axis=1).sum(axis=1)

In [ ]:
twkm.filter(regex="H2", axis=1).sum(axis=1)

In [ ]:
322.2 * 1.25

In [ ]:
758.6 / 692.3

In [ ]:
758.6 / 322.2

In [ ]:
758.6 / 322.2

In [ ]:
692.3 / 322.2

In [ ]:
twkm

In [ ]:
twkm.filter(like="H2", axis=1).sum(axis=1)

## Capacities

In [ ]:
df = load_h2_capacities().xs(100.0, axis=1, level="onw").div(1e3)  # GW

In [ ]:
index_dict = {
    "offwind-ac": "offshore wind (AC)",
    "offwind-dc": "offshore wind (DC)",
    "onwind": "onshore wind",
}

column_dict = {
    "1.0": "w/o power",
    "opt": "w power",
    "H2 grid": "w H$_2$",
    "no H2 grid": "w/o H$_2$",
}

df.rename(index=index_dict, columns=column_dict, inplace=True)

In [ ]:
df.columns = ["\n".join(col).strip() for col in df.columns.values]

In [ ]:
df.sort_index(axis=1, inplace=True)

In [ ]:
tech_colors = config["plotting"]["tech_colors"]

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(10, 4))

groups = [
    ["solar rooftop", "solar"],
    ["onshore wind"],
    ["offshore wind (AC)", "offshore wind (DC)"],
    ["H2 Electrolysis", "SMR CC", "SMR"],
]

ylims = [
    [0, 4000],
    [0, 2000],
    [0, 300],
    [0, 1500],
]

for ax, group, ylim in zip(axs, groups, ylims):
    df.loc[group].T.plot.bar(ax=ax, stacked=True, color=tech_colors)
    # ax.set_xlabel('grid expansion')
    ax.legend(bbox_to_anchor=(1.02, 1.45))
    ax.set_ylabel("capacity [GW]")
    ax.set_ylim(ylim)

# fig.supxlabel('grid expansion')
plt.tight_layout()
plt.savefig(OUTPUT + "capacities.pdf", bbox_inches="tight")